In [ ]:
import phoenix as px
from llama_index import Document, ServiceContext, SummaryIndex
from llama_index.callbacks import CallbackManager
from llama_index.indices.list import SummaryIndexRetriever
from llama_index.llms import OpenAI
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.response_synthesizers import CompactAndRefine
from phoenix.trace.llama_index import (
    OpenInferenceTraceCallbackHandler,
)
from pydantic import BaseModel, Field

session = px.launch_app()

callback_handler = OpenInferenceTraceCallbackHandler()
cb_manager = CallbackManager(handlers=[callback_handler])

service_ctx = ServiceContext.from_defaults(
    llm=(OpenAI(model="gpt-3.5-turbo-16k", max_retries=10, callback_manager=cb_manager)),
    callback_manager=cb_manager,
)


class Person(BaseModel):
    """A class representing a person mentioned in the text"""

    name: str = Field(description="The person's name")
    age: int | None = Field(description="The person's age")


class People(BaseModel):
    """A class representing a collection of unique individuals"""

    people: list[Person] = Field(description="A list of all unique people mentioned in a text.")


from textwrap import dedent

doc = Document(
    text=dedent(
        """
        Bobby Shaftoe was only 16 when he went to sea as a deckhand. \
        He fell victim to the scurvy after just 2 months. \
        Unfazed, Captain Jack discharged his duties with extreme prejudice. \
        """
    )
)

response_synth = CompactAndRefine(
    service_context=service_ctx,
)

query_eng = RetrieverQueryEngine(
    retriever=SummaryIndexRetriever(index=SummaryIndex.from_documents([doc])),
    response_synthesizer=response_synth,
    callback_manager=cb_manager,
)

query_eng.query("Who is mentioned in this contrived, minimal 'story'?")

df = session.get_spans_dataframe()

In [ ]:
df.count()